APIキーを定義、パッケージimport、関数を定義します。

In [14]:
%env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [2]:
%pip install openai langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
import concurrent.futures
import os

import openai
from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool
from langchain.prompts import MessagesPlaceholder

from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

現在時刻を取得する関数でfunction callingを検証します。

In [9]:
import ntplib
import pytz
from datetime import datetime, timezone

def fetch_current_datetime_in_timezone(timezone_str):
    client = ntplib.NTPClient()
    response = client.request('pool.ntp.org')
    utc_now = datetime.fromtimestamp(response.tx_time, timezone.utc)
    try:
        target_timezone = pytz.timezone(timezone_str)
        return utc_now.astimezone(target_timezone)
    except pytz.UnknownTimeZoneError:
        raise ValueError(f"Unsupported timezone: {timezone_str}")

class TimeZoneDatetimeInput(BaseModel):
  timezone_str: str = Field(description="Time zone (e.g., Asia/Tokyo, UTC, America/New_York, Europe/London, etc.)")

class TimeZoneDatetimeFetcher(BaseTool):
    name = 'fetch_datetime_in_timezone'
    description = "Enter time zone (e.g., Asia/Tokyo, UTC, America/New_York, Europe/London, etc.). Outputs the current date and time corresponding to the time zone."
    args_schema: Type[BaseModel] = TimeZoneDatetimeInput
    
    def _run(self, timezone_str: str):
        return fetch_current_datetime_in_timezone(timezone_str)
    
    def _arun(self, ticker: str):
        raise NotImplementedError("fetch_datetime_in_timezone does not support async")

In [11]:
from function_calling_tools.ntp import TimeZoneDatetimeFetcher

tools = [
  TimeZoneDatetimeFetcher(),
]

memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

agent_kwargs = {
  "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}

prompt = """
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
"""

memory.save_context({"input": prompt}, {"output": "I got it!"})

openai.api_key = os.environ["OPENAI_API_KEY"]
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)

前準備が終わったら、あとは質問するのみです。

In [12]:
agent.run(input="現在時刻は何時ですか？")



> Entering new AgentExecutor chain...

Invoking: `fetch_datetime_in_timezone` with `{'timezone_str': 'Asia/Tokyo'}`


2023-09-11 12:27:51.333530+09:00現在の時刻は、2023年9月11日 12時27分です。

> Finished chain.


'現在の時刻は、2023年9月11日 12時27分です。'

In [13]:
agent.run(input="今週の月曜から日曜まで、それぞれ何月何日が出力してほしい")



> Entering new AgentExecutor chain...

Invoking: `fetch_datetime_in_timezone` with `{'timezone_str': 'Asia/Tokyo'}`


2023-09-11 12:27:56.949221+09:00現在の日付は2023年9月11日です。以下は今週の月曜日から日曜日までの日付です。

- 月曜日: 2023年9月11日
- 火曜日: 2023年9月12日
- 水曜日: 2023年9月13日
- 木曜日: 2023年9月14日
- 金曜日: 2023年9月15日
- 土曜日: 2023年9月16日
- 日曜日: 2023年9月17日

どうぞご参考ください。

> Finished chain.


'現在の日付は2023年9月11日です。以下は今週の月曜日から日曜日までの日付です。\n\n- 月曜日: 2023年9月11日\n- 火曜日: 2023年9月12日\n- 水曜日: 2023年9月13日\n- 木曜日: 2023年9月14日\n- 金曜日: 2023年9月15日\n- 土曜日: 2023年9月16日\n- 日曜日: 2023年9月17日\n\nどうぞご参考ください。'